In [4]:
import random
import pandas as pd
from datetime import datetime, timedelta

def generate_random_date(start_date, end_date):
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

def generate_report(building_name, date):
    return {
        "Building": building_name,
        "Inspection_Date": date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3],
        "Roof_Status": random.randint(1, 5),
        "Roof_Issue": random.randint(0, 5),
        "Siding_Status": random.randint(1, 5),
        "Siding_Issue": random.randint(0, 5),
        "Windows_Status": random.randint(1, 5),
        "Windows_Issue": random.randint(0, 5),
        "HVAC_Status": random.randint(1, 5),
        "HVAC_Issue": random.randint(0, 5),
        "Plumbing_Status": random.randint(1, 5),
        "Plumbing_Issue": random.randint(0, 5),
        "Elevators_Status": random.randint(1, 5),
        "Elevators_Issue": random.randint(0, 5),
        "Lighting_Status": random.randint(1, 5),
        "Lighting_Issue": random.randint(0, 5),
        "Landscaping_Status": random.randint(1, 5),
        "Landscaping_Issue": random.randint(0, 5),
        "Emergency Exits_Status": random.randint(1, 5),
        "Emergency Exits_Issue": random.randint(0, 5)
    }

def generate_building_reports(building_name, num_reports, start_date, end_date):
    reports = []
    for _ in range(num_reports):
        date = generate_random_date(start_date, end_date)
        reports.append(generate_report(building_name, date))
    return reports

def generate_dataset(num_buildings, num_reports_per_building):
    buildings = [f"Building {chr(65 + i)}" for i in range(num_buildings)]
    start_date = datetime(2020, 1, 1)
    end_date = datetime(2026, 12, 31)
    
    dataset = []
    for building in buildings:
        dataset.extend(generate_building_reports(building, num_reports_per_building, start_date, end_date))
    
    return dataset

# Generate dataset
dataset = generate_dataset(10, 5)  # 10 buildings, 5 reports each

# Convert to DataFrame
df = pd.DataFrame(dataset)

# Save to CSV
df.to_csv('building_inspection_reports.csv', index=False)

print("CSV file 'building_inspection_reports.csv' has been created.")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv('building_inspection_reports.csv')

# Convert Inspection_Date to datetime
df['Inspection_Date'] = pd.to_datetime(df['Inspection_Date'])

# Feature engineering: Extract year, month, and day from Inspection_Date
df['Year'] = df['Inspection_Date'].dt.year
df['Month'] = df['Inspection_Date'].dt.month
df['Day'] = df['Inspection_Date'].dt.day

# Drop the original Inspection_Date column
df = df.drop(columns=['Inspection_Date'])

# Encode the Building column
df = pd.get_dummies(df, columns=['Building'])

# Define features and target variable
X = df.drop(columns=['Roof_Status'])
y = df['Roof_Status']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Function to predict future maintenance status
def predict_future_maintenance(building, date):
    date = pd.to_datetime(date)
    year, month, day = date.year, date.month, date.day
    building_encoded = [1 if f'Building_{building}' in col else 0 for col in X.columns if col.startswith('Building_')]
    features = [year, month, day] + building_encoded
    
    # Ensure the feature vector has the same length as the training data
    while len(features) < len(X.columns):
        features.append(0)
    
    return model.predict([features])[0]

# Example prediction
future_date = '2027-01-01T00:00:00.000'
building = 'Building A'
predicted_roof_status = predict_future_maintenance(building, future_date)
print(f"Predicted Roof Status for {building} on {future_date}: {predicted_roof_status}")

Mean Squared Error: 1.93106
Predicted Roof Status for Building A on 2027-01-01T00:00:00.000: 2.83


c:\Users\HP\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the dataset
df = pd.read_csv('building_inspection_reports.csv')

# Convert Inspection_Date to datetime
df['Inspection_Date'] = pd.to_datetime(df['Inspection_Date'])

# Feature engineering: Extract year, month, and day from Inspection_Date
df['Year'] = df['Inspection_Date'].dt.year
df['Month'] = df['Inspection_Date'].dt.month
df['Day'] = df['Inspection_Date'].dt.day

# Drop the original Inspection_Date column
df = df.drop(columns=['Inspection_Date'])

# Encode the Building column
df = pd.get_dummies(df, columns=['Building'])

# Define features and target variables
features = df.drop(columns=[
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
])
target_columns = [
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
]

# Convert target variables to binary (1 if maintenance is required, 0 otherwise)
for col in target_columns:
    df[col] = df[col].apply(lambda x: 1 if x < 3 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, df[target_columns], test_size=0.2, random_state=42)

# Initialize and train the models for each target variable
models = {}
for target in target_columns:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[target])
    models[target] = model

# Function to predict future maintenance requirement
def predict_future_maintenance(building, date):
    date = pd.to_datetime(date)
    year, month, day = date.year, date.month, date.day
    building_encoded = [1 if f'Building_{building}' in col else 0 for col in features.columns if col.startswith('Building_')]
    input_features = [year, month, day] + building_encoded
    
    # Ensure the feature vector has the same length as the training data
    while len(input_features) < len(features.columns):
        input_features.append(0)
    
    predictions = {}
    for target, model in models.items():
        predictions[target] = model.predict([input_features])[0]
    
    return predictions

# Example prediction
future_date = '2027-01-01T00:00:00.000'
building = 'Building A'
predictions = predict_future_maintenance(building, future_date)

# Print detailed analysis
print(f"Predicted Maintenance Requirement for {building} on {future_date}:")
for param, value in predictions.items():
    status = "Maintenance Required" if value == 1 else "No Maintenance Required"
    print(f"{param}: {status}")

# Evaluate the models
for target in target_columns:
    y_pred = models[target].predict(X_test)
    print(f"Classification Report for {target}:")
    print(classification_report(y_test[target], y_pred))

Predicted Maintenance Requirement for Building A on 2027-01-01T00:00:00.000:
Roof_Status: Maintenance Required
Roof_Issue: No Maintenance Required
Siding_Status: No Maintenance Required
Siding_Issue: No Maintenance Required
Windows_Status: Maintenance Required
Windows_Issue: No Maintenance Required
HVAC_Status: No Maintenance Required
HVAC_Issue: No Maintenance Required
Plumbing_Status: No Maintenance Required
Plumbing_Issue: No Maintenance Required
Elevators_Status: No Maintenance Required
Elevators_Issue: No Maintenance Required
Lighting_Status: No Maintenance Required
Lighting_Issue: No Maintenance Required
Landscaping_Status: No Maintenance Required
Landscaping_Issue: No Maintenance Required
Emergency Exits_Status: No Maintenance Required
Emergency Exits_Issue: Maintenance Required
Classification Report for Roof_Status:
              precision    recall  f1-score   support

           0       0.56      0.83      0.67         6
           1       0.00      0.00      0.00         4



c:\Users\HP\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\GPU\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\GPU

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

# Load the dataset
df = pd.read_csv('building_inspection_reports.csv')

# Convert Inspection_Date to datetime
df['Inspection_Date'] = pd.to_datetime(df['Inspection_Date'])

# Feature engineering: Extract year, month, and day from Inspection_Date
df['Year'] = df['Inspection_Date'].dt.year
df['Month'] = df['Inspection_Date'].dt.month
df['Day'] = df['Inspection_Date'].dt.day

# Drop the original Inspection_Date column
df = df.drop(columns=['Inspection_Date'])

# Encode the Building column
df = pd.get_dummies(df, columns=['Building'])

# Define features and target variables
features = df.drop(columns=[
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
])
target_columns = [
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
]

# Convert target variables to binary (1 if maintenance is required, 0 otherwise)
for col in target_columns:
    df[col] = df[col].apply(lambda x: 1 if x < 3 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, df[target_columns], test_size=0.2, random_state=42)

# Initialize and train the models for each target variable
models = {}
for target in target_columns:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[target])
    models[target] = model

# Function to predict future maintenance requirement
def predict_future_maintenance(building, date):
    date = pd.to_datetime(date)
    year, month, day = date.year, date.month, date.day
    building_encoded = [1 if f'Building_{building}' in col else 0 for col in features.columns if col.startswith('Building_')]
    input_features = [year, month, day] + building_encoded
    
    # Ensure the feature vector has the same length as the training data
    while len(input_features) < len(features.columns):
        input_features.append(0)
    
    predictions = {}
    for target, model in models.items():
        predictions[target] = model.predict_proba([input_features])[0][1]  # Probability of maintenance required
    
    return predictions

# Function to find the future date with the highest probability of maintenance required
def find_future_maintenance_date(building, start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date)
    max_prob = 0
    maintenance_date = None
    for date in date_range:
        predictions = predict_future_maintenance(building, date)
        avg_prob = np.mean(list(predictions.values()))
        if avg_prob > max_prob:
            max_prob = avg_prob
            maintenance_date = date
    return maintenance_date, max_prob

# Example prediction
future_date = '2027-01-01T00:00:00.000'
building = 'Building A'
predictions = predict_future_maintenance(building, future_date)

# Print detailed analysis
print(f"Predicted Maintenance Requirement for {building} on {future_date}:")
for param, value in predictions.items():
    status = "Maintenance Required" if value >= 0.5 else "No Maintenance Required"
    print(f"{param}: {status} (Probability: {value:.2f})")

# Find the future date with the highest probability of maintenance required
start_date = '2027-01-01'
end_date = '2027-12-31'
maintenance_date, max_prob = find_future_maintenance_date(building, start_date, end_date)
print(f"Future date with highest probability of maintenance required for {building}: {maintenance_date} (Probability: {max_prob:.2f})")

# Show in table what all is required for maintenance
predictions = predict_future_maintenance(building, maintenance_date)
print(f"\nMaintenance requirements for {building} on {maintenance_date}:")
print(f"{'Component':<25} {'Maintenance Required':<20} {'Probability':<10}")
print("-" * 60)
for param, value in predictions.items():
    status = "Yes" if value >= 0.5 else "No"
    print(f"{param:<25} {status:<20} {value:.2f}")

# Evaluate the models
for target in target_columns:
    y_pred = models[target].predict(X_test)
    print(f"Classification Report for {target}:")
    print(classification_report(y_test[target], y_pred))

c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda

Predicted Maintenance Requirement for Building A on 2027-01-01T00:00:00.000:
Roof_Status: Maintenance Required (Probability: 0.55)
Roof_Issue: No Maintenance Required (Probability: 0.19)
Siding_Status: No Maintenance Required (Probability: 0.11)
Siding_Issue: No Maintenance Required (Probability: 0.21)
Windows_Status: Maintenance Required (Probability: 0.77)
Windows_Issue: No Maintenance Required (Probability: 0.41)
HVAC_Status: No Maintenance Required (Probability: 0.43)
HVAC_Issue: No Maintenance Required (Probability: 0.19)
Plumbing_Status: No Maintenance Required (Probability: 0.30)
Plumbing_Issue: No Maintenance Required (Probability: 0.06)
Elevators_Status: No Maintenance Required (Probability: 0.42)
Elevators_Issue: No Maintenance Required (Probability: 0.28)
Lighting_Status: No Maintenance Required (Probability: 0.21)
Lighting_Issue: Maintenance Required (Probability: 0.50)
Landscaping_Status: No Maintenance Required (Probability: 0.07)
Landscaping_Issue: No Maintenance Require

c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda

Future date with highest probability of maintenance required for Building A: 2027-03-30 00:00:00 (Probability: 0.47)

Maintenance requirements for Building A on 2027-03-30 00:00:00:
Component                 Maintenance Required Probability
------------------------------------------------------------
Roof_Status               Yes                  0.56
Roof_Issue                No                   0.31
Siding_Status             No                   0.38
Siding_Issue              No                   0.08
Windows_Status            No                   0.37
Windows_Issue             Yes                  0.63
HVAC_Status               Yes                  0.55
HVAC_Issue                Yes                  0.64
Plumbing_Status           Yes                  0.75
Plumbing_Issue            No                   0.26
Elevators_Status          Yes                  0.63
Elevators_Issue           Yes                  0.57
Lighting_Status           No                   0.41
Lighting_Issue        

c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
import numpy as np

# Load the dataset
df = pd.read_csv('building_inspection_reports.csv')

# Convert Inspection_Date to datetime
df['Inspection_Date'] = pd.to_datetime(df['Inspection_Date'])

# Feature engineering: Extract year, month, and day from Inspection_Date
df['Year'] = df['Inspection_Date'].dt.year
df['Month'] = df['Inspection_Date'].dt.month
df['Day'] = df['Inspection_Date'].dt.day

# Drop the original Inspection_Date column
df = df.drop(columns=['Inspection_Date'])

# Encode the Building column
df = pd.get_dummies(df, columns=['Building'])

# Define features and target variables
features = df.drop(columns=[
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
])
target_columns = [
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
]

# Convert target variables to binary (1 if maintenance is required, 0 otherwise)
for col in target_columns:
    df[col] = df[col].apply(lambda x: 1 if x < 3 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, df[target_columns], test_size=0.2, random_state=42)

# Initialize and train the models for each target variable
models = {}
for target in target_columns:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[target])
    models[target] = model

# Save the combined model and feature columns
joblib.dump((models, features.columns), 'combined_model.joblib')

print("Combined model has been saved.")

Combined model has been saved.


In [11]:
import pandas as pd
import joblib
import numpy as np

# Load the dataset
df = pd.read_csv('building_inspection_reports.csv')

# Convert Inspection_Date to datetime
df['Inspection_Date'] = pd.to_datetime(df['Inspection_Date'])

# Feature engineering: Extract year, month, and day from Inspection_Date
df['Year'] = df['Inspection_Date'].dt.year
df['Month'] = df['Inspection_Date'].dt.month
df['Day'] = df['Inspection_Date'].dt.day

# Drop the original Inspection_Date column
df = df.drop(columns=['Inspection_Date'])

# Encode the Building column
df = pd.get_dummies(df, columns=['Building'])

# Define features
features = df.drop(columns=[
    'Roof_Status', 'Roof_Issue', 'Siding_Status', 'Siding_Issue', 'Windows_Status', 'Windows_Issue',
    'HVAC_Status', 'HVAC_Issue', 'Plumbing_Status', 'Plumbing_Issue', 'Elevators_Status', 'Elevators_Issue',
    'Lighting_Status', 'Lighting_Issue', 'Landscaping_Status', 'Landscaping_Issue', 'Emergency Exits_Status', 'Emergency Exits_Issue'
])

# Load the combined model and feature columns
models, feature_columns = joblib.load('combined_model.joblib')

# Function to predict future maintenance requirement
def predict_future_maintenance(building, date):
    date = pd.to_datetime(date)
    year, month, day = date.year, date.month, date.day
    building_encoded = [1 if f'Building_{building}' in col else 0 for col in feature_columns if col.startswith('Building_')]
    input_features = [year, month, day] + building_encoded
    
    # Ensure the feature vector has the same length as the training data
    while len(input_features) < len(feature_columns):
        input_features.append(0)
    
    predictions = {}
    for target, model in models.items():
        predictions[target] = model.predict_proba([input_features])[0][1]  # Probability of maintenance required
    
    return predictions

# Example prediction
future_date = '2027-01-01T00:00:00.000'
building = 'Building A'
predictions = predict_future_maintenance(building, future_date)

# Print detailed analysis
print(f"Predicted Maintenance Requirement for {building} on {future_date}:")
for param, value in predictions.items():
    status = "Maintenance Required" if value >= 0.5 else "No Maintenance Required"
    print(f"{param}: {status} (Probability: {value:.2f})")

Predicted Maintenance Requirement for Building A on 2027-01-01T00:00:00.000:
Roof_Status: Maintenance Required (Probability: 0.55)
Roof_Issue: No Maintenance Required (Probability: 0.19)
Siding_Status: No Maintenance Required (Probability: 0.11)
Siding_Issue: No Maintenance Required (Probability: 0.21)
Windows_Status: Maintenance Required (Probability: 0.77)
Windows_Issue: No Maintenance Required (Probability: 0.41)
HVAC_Status: No Maintenance Required (Probability: 0.43)
HVAC_Issue: No Maintenance Required (Probability: 0.19)
Plumbing_Status: No Maintenance Required (Probability: 0.30)
Plumbing_Issue: No Maintenance Required (Probability: 0.06)
Elevators_Status: No Maintenance Required (Probability: 0.42)
Elevators_Issue: No Maintenance Required (Probability: 0.28)
Lighting_Status: No Maintenance Required (Probability: 0.21)
Lighting_Issue: Maintenance Required (Probability: 0.50)
Landscaping_Status: No Maintenance Required (Probability: 0.07)
Landscaping_Issue: No Maintenance Require

c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda3\envs\torch\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\HP\anaconda

In [ ]:
pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.2 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.2 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.2 MB 508.0 kB/s eta 0:00:06
   ------ --------------------------------- 0.5/3.2 MB 508.0 kB/s eta 0:00:06
   --------- ------------------------------ 0.8/3.2 MB 610.3 kB/s eta 0:00:04
   --------- ------------------------------ 0.8/3.2 MB 610.3 kB/s eta 0:00:04
   ------------- -------------------------- 1.0/3.2 MB 592.2 kB/s eta 0:00:04
   ------------- -------------------------- 1.0/3.2 MB 592.2 kB/s eta 0:00:04
   ---------------- ----------------------- 1

In [12]:
import tkinter as tk
from tkinter import filedialog
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import fitz  # PyMuPDF
import pandas as pd

def read_pdf_report(filename):
    doc = fitz.open(filename)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def analyze_maintenance_report(text):
    lines = text.split('\n')
    maintenance_data = {}
    for line in lines:
        if ": " in line:
            key, value = line.split(": ", 1)
            maintenance_data[key] = value
    return maintenance_data

def generate_analysis_report(maintenance_data, output_filename):
    c = canvas.Canvas(output_filename, pagesize=letter)
    width, height = letter

    # Title
    c.setFont("Helvetica-Bold", 20)
    c.drawString(100, height - 50, "Maintenance Analysis Report")

    # Add maintenance data to the PDF
    y_position = height - 100
    for key, value in maintenance_data.items():
        c.setFont("Helvetica", 12)
        c.drawString(100, y_position, f"{key}: {value}")
        y_position -= 20

    # Save the PDF
    c.save()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
    return file_path

# Select the PDF file using file picker
selected_file = select_file()
if selected_file:
    # Read and analyze the PDF report
    text = read_pdf_report(selected_file)
    extracted_data = analyze_maintenance_report(text)

    print("Extracted Maintenance Data:")
    for key, value in extracted_data.items():
        print(f"{key}: {value}")

    # Generate analysis report in PDF format
    output_filename = "maintenance_analysis_report.pdf"
    generate_analysis_report(extracted_data, output_filename)
    print(f"Analysis report '{output_filename}' has been created.")
else:
    print("No file selected.")

In [11]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.6 MB 2.4 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.6 MB 2.3 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/11.6 MB 1.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/11.6 MB 1.7 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.6 MB 1.4 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.6 MB 1.4 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.6 MB 1.1 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/11.6 MB 1.1 MB/s eta 0:00:09
   -------- -----------------------